<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [67]:
import pandas as pd  
import psycopg2
import requests
from bs4 import BeautifulSoup

In [68]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

In [69]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [70]:
# Убираем лишние собщения в выводе.
import warnings
warnings.filterwarnings("ignore") 

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [71]:
# текст запроса
query_3_1 = f'''

    SELECT COUNT(id) as count
    FROM VACANCIES  
    
'''

In [72]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
print(f'Количество вакансий: {df.iloc[0,0]}') 

Количество вакансий: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [73]:
# текст запроса
query_3_2 = f'''

    SELECT COUNT(id)
    FROM employers  
    
'''

In [74]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
print(f'Количество работодателей: {df.iloc[0,0]}')

Количество работодателей: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [75]:
# текст запроса
query_3_3 = f'''

    SELECT COUNT(DISTINCT id)
    FROM areas     
    
'''

In [76]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
print(f'Количество регионов: {df.iloc[0,0]}')

Количество регионов: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [77]:
# текст запроса
query_3_4 = f'''

    SELECT COUNT(id)
    FROM industries
     
'''

In [78]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
print(f'Количество сфер деятельности в базе: {df.iloc[0,0]}')

Количество сфер деятельности в базе: 294


***

# Выводы по предварительному анализу данных

- В базе вакансий содержится 49197 вакансий;
- Вакансии разместили 23501 работодателя;
- Вакансии имеются в 1362 регионе; 
- Общее число сфер деятельности работодателей 294.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [79]:
# текст запроса
query_4_1 = f'''

    SELECT 
        a.name area,
        count(a.id) cnt    
    FROM VACANCIES v
    left join AREAS a ON a.id = v.area_id
    GROUP BY a.name
    ORDER BY cnt DESC

'''

In [80]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df.head(5) 

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [81]:
# текст запроса
query_4_2 = f'''

    SELECT COUNT (id)    
    FROM VACANCIES 
    WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
   
'''

In [82]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий где заполнено хотя бы одно из двух полей с зарплатой: {df.iloc[0,0]}')

Количество вакансий где заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [83]:
# текст запроса

query_4_3 = f'''

    /* Округляем до целого среднюю нижнюю и верхню границу зарплаты и приводим к типу int.*/
    SELECT 
        ROUND(AVG(salary_from))::int avg_min, 
        ROUND(AVG(salary_to))::int avg_max     
    FROM VACANCIES
    
'''

In [84]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
print(f'Среднее значение для нижней и верхней границы зарплатной вилки: {df.iloc[0,0]} и {df.iloc[0,1]} соответственно.')

Среднее значение для нижней и верхней границы зарплатной вилки: 71065 и 110537 соответственно.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [85]:
# текст запроса
query_4_4 = f'''

    /*Считаем количество вакансий по критериям сочетания типа рабочего графика и типа трудоустройства.*/
    SELECT 
        COUNT(id) cnt, 
        schedule тип_рабочего_графика ,
        employment тип_трудоустройства             
    FROM VACANCIES
    GROUP BY 
        schedule,
        employment
    ORDER BY cnt DESC
        
'''

In [86]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,cnt,тип_рабочего_графика,тип_трудоустройства
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [87]:
# текст запроса
query_4_5 = f'''

    /* Запрос выводит поле с требуемым опытом и соответствующее ему количество вакансий*/
    select 
        experience as experience,
        count(id) as count
        
    from VACANCIES
    group by experience
    order by count asc
   
'''

In [88]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Выводы по детальному анализу вакансий
- Из данных следует, что наибольшее количество вакансий представлено в Москве, затем, с отрывом более 
  2000 вакансий идет Санкт-Петербург, что видимо с вязано с повышенной экономической активностью в них.

- Работодатели не всегда (24073 из 49197 вакансий) указывают пределы ЗП для соискателей,
  что означает их готовность обсуждать с соискателями её размер индивидуально.

- Основная масса работодателей предлагают ЗП в пределах 71000 - 110000.  

- Максимум вакансий содержат требования к типу рабочего графика -  полный день с 
  типом трудоустройства - полная занятость.

- Показатель распределения количества вакансий от опыта поззволяет сделать вывод, 
  что специалистов с опытом работы более 6     лет  

- Максимум вакансий предлагается для соискателей с опытом от 1 года до 3. 
  Видимо здесь наблюдается наибольшая 'текучка' кадров. Вторым, по количеству вакансий, 
  идет опыт от 3 до 6 лет, где уже требования к соискателям более высокие. 
  Специалистов с опытом более 6 лет компании компании видимо стремятся удержать 
  из-за их компетенций - от того и количество вакансий минимально. 



***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [89]:
# текст запроса
query_5_1 = f'''
    
   (
      SELECT   /*Запрос выводящий первого работодателя по количеству вакансий.*/
         e.name company_name,
         count(e.id) cnt       
      FROM VACANCIES v
      JOIN EMPLOYERS e ON v.employer_id = e.id
      GROUP BY e.id
      ORDER BY cnt DESC
      LIMIT 1
   )
      
   UNION  /*Соединяем результаты запросов*/
      
   (
      SELECT   /*Запрос выводящий пятого работодателя по количеству вакансий.*/
         e.name company_name,
         COUNT(e.id) cnt       
      FROM VACANCIES v
      JOIN EMPLOYERS e ON v.employer_id = e.id
      GROUP BY e.id
      ORDER BY cnt DESC
      OFFSET 4
      LIMIT 1
   )
   ORDER BY cnt DESC  /*Группируем по количеству вакансий.*/ 

'''

In [90]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,company_name,cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [91]:
# текст запроса
query_5_2 = f'''

    SELECT /*Запрос выыодит регион с наибольшим количеством работодателей с отсутствующими вакансиями.*/
        a.name area,        
        COUNT(DISTINCT e.id) employer_cnt,
        COUNT(v.id) vacancy_cnt
    FROM 
        AREAS a
        LEFT JOIN VACANCIES v ON a.id = v.area_id         
        LEFT JOIN EMPLOYERS e ON a.id = e.area
    WHERE v.id IS NULL  
    GROUP BY a.name  
    ORDER BY employer_cnt DESC
    LIMIT 1
    
'''


In [92]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
print(f'Регион с наибольшим количеством работодателей ({df.iloc[0,1]}), в котором нет вакансий: {df.iloc[0,0]}.')

Регион с наибольшим количеством работодателей (410), в котором нет вакансий: Россия.


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [93]:
# текст запроса
query_5_3 = f'''

    SELECT /*Запрос выводит название работодателя и число регионов представления его вакансий.*/
        e.name employer,  
        COUNT(DISTINCT v.area_id) areas_cnt 
    FROM EMPLOYERS e
        LEFT JOIN VACANCIES v ON e.id = v.employer_id
    GROUP BY e.name   
    ORDER BY areas_cnt DESC 
   
    
'''

In [94]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
display(df)
print(f'Работодатель с наибольшим числом регионов присутствия - {df.iloc[0,0]}, представлен в {df.iloc[0,1]} регионе.')

,employer,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


Работодатель с наибольшим числом регионов присутствия - Яндекс, представлен в 181 регионе.


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [95]:
# текст запроса
query_5_4 = f'''

    SELECT /*Запрос выводит количество работодателей неуказавших сферу деятельности.*/
        COUNT (e.id)        
    FROM EMPLOYERS e
        LEFT JOIN EMPLOYERS_INDUSTRIES em ON e.id = em.employer_id
        LEFT JOIN INDUSTRIES i ON em.industry_id = i.id 
    WHERE i.name IS NULL       
'''
    

In [96]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
print(f'Количество работодателей, у которых не указана сфера деятельности: {df.iloc[0,0]}.')

Количество работодателей, у которых не указана сфера деятельности: 8419.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [97]:
# текст запроса
query_5_5 = f'''

    SELECT /*Запрос выводит название компании находящейся на 3 месте среди компаний с 4 сферами деятельности*/
        e.name employer,        
        COUNT(DISTINCT i.id) industries_cnt        
    FROM EMPLOYERS e
        LEFT JOIN EMPLOYERS_INDUSTRIES em ON e.id = em.employer_id
        LEFT JOIN INDUSTRIES i ON em.industry_id = i.id 
    /*Отсеиваем компании, не указавших сферы деятельности (уменьшаем время выполнения запроса)*/
    WHERE i.name IS NOT NULL    
    GROUP BY e.name 
    HAVING COUNT(DISTINCT i.id) = 4
    ORDER BY e.name ASC 
    OFFSET 2
    LIMIT 1 
    
'''
    

In [98]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
print(f'Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний, \
у которых указано четыре сферы деятельности: {df.iloc[0,0]}.')

Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности: 2ГИС.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [99]:
# текст запроса
query_5_6 = f'''

    SELECT /*Количество работодателей со сферой деятельности - Разработка программного обеспечения. */
        COUNT(employer_id)
    FROM EMPLOYERS e
        LEFT JOIN EMPLOYERS_INDUSTRIES em ON e.id = em.employer_id
        LEFT JOIN INDUSTRIES i ON i.id = em.industry_id 
    WHERE i.name = 'Разработка программного обеспечения'  
      
'''

In [100]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
print(f'Разработкой программного обеспечения занимаются: {df.iloc[0,0]} работодателя.')

Разработкой программного обеспечения занимаются: 3553 работодателя.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [101]:
# код для получения списка городов-милионников
list_m_city = []
URL = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(URL)
page = BeautifulSoup(response.text, 'html.parser')
# получаем строку с тегом 'table' класс 'standard' содержащую всю информацию в таблице
temp_str = str(page.find('table', class_='standard')).split('title="') 
# добавляем в список только названия городов
for i in temp_str[3:]:
    temp = i.split('">')[0]
    list_m_city.append(temp)

In [102]:
# текст запроса
query_5_7 = f'''

    /*Создаем таблицу temp_table для подсчета количества вакансий компании «Яндекс» в городах-миллионниках*/
    WITH temp_table AS
        (
            SELECT 
                a.name,
                COUNT(v.id)::int cnt  /*Количество вакансий для лучшего отображения приводим к типу int.*/
            FROM AREAS a 
            LEFT JOIN VACANCIES v on v.area_id = a.id
            LEFT JOIN EMPLOYERS e on e.id = v.employer_id   
            /*Ищем вакансии компании только в списке городов-миллионников представленном в виде кортежа.*/
            WHERE e.name = 'Яндекс' AND a.name IN {tuple(list_m_city)}
            GROUP BY a.id
            ORDER BY cnt
        )
    SELECT /*Выводим название города и количество вакансий компании.*/
       name,
       cnt
    FROM temp_table 
    UNION /*Присоединяем строку Total с общим количеством вакансий в этих городах*/
    SELECT 
        'Total',
        SUM(cnt) cnt 
    FROM temp_table 
    ORDER BY cnt  /*Сортируем по возрастанию количества вакансий*/
      
'''

In [103]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


# Выводы по анализу работодателей
- По результату проведенного исследования видно что топ-10 компаний по количеству вакансий - IT компании, банки и Газпром нефть.

 - Имеются регионы где представлено много работодателей, но отсутствуют вакансии - видимо это связано с ошибкой размещения вакансий.

- Максимальное число регионов размещения вакансий одним работодателем - 181, что указывает на его обширный масштаб деятельности.

- Масимальное число сфер деятельности работодателей - 16, при этом, число работодателей не указавших сфер деятельности - 8419.
  
- Разработкой программного обеспечения занимаются: 3553 работодателя.

- Для компании «Яндекс» из 1933 вакансий 485 представлены в городах-миллионниках

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [104]:
# текст запроса
query_6_1 = f'''
 
    SELECT /*Запрос выводит количество вакансий имеющих отношение к данным.*/
        COUNT(id)
    FROM 
        VACANCIES
    WHERE 
       name ILIKE '%data%' or name ILIKE '%данн%' 
    
'''

In [105]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
print(f'Количество вакансий имеющих отношение к данным: {df.iloc[0,0]}.')

Количество вакансий имеющих отношение к данным: 1771.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [106]:
# текст запроса
query_6_2 = f''' 

    SELECT /*Запрос выводит количество вакансий для начинающего дата-сайентиста.*/
        COUNT(DISTINCT id)
    FROM 
        VACANCIES 
    WHERE (name ILIKE '%data scientist%' 
           OR name ILIKE '%data science%' 
           OR name ILIKE '%исследователь данных%' 
           OR name ILIKE '%машинн%обучен%%'
           OR name ILIKE '%machine learning%'
           OR (name LIKE '%ML%' 
               AND name NOT LIKE '%HTML%'))
           AND (name ILIKE '%junior%' 
                OR experience ='Нет опыта'
                OR schedule = 'Стажировка') 
        
'''

In [107]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
print(f'Количество подходящих вакансий для начинающего дата-сайентиста: {df.iloc[0,0]}.')

Количество подходящих вакансий для начинающего дата-сайентиста: 51.


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [108]:
# текст запроса
query_6_3 = f'''

    /*Запрос выводит количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres*/
    SELECT
        count(id)
    FROM  
        VACANCIES 
    WHERE (name ILIKE '%data scientist%' 
        OR name ILIKE '%data science%' 
        OR name ILIKE '%исследователь данных%' 
        OR name ILIKE '%машинн%обучен%%'
        OR name ILIKE '%machine learning%'
        OR (name ILIKE '%ML%' AND name NOT LIKE '%HTML%' and name not like '%QML%'))
    and (key_skills ILIKE'%SQL%' 
        OR key_skills ILIKE '%postgres%')
 
'''

In [109]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df
print(f'Количество вакансий c ключевым навыком SQL или postgres: {df.iloc[0,0]}.')

Количество вакансий c ключевым навыком SQL или postgres: 229.


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [110]:
# текст запроса
query_6_4 = f'''

    SELECT /*Запрос выводит количество вакансий c ключевым навыком Python.*/
        count(id)
    FROM  
        VACANCIES 
    WHERE (name ILIKE '%data scientist%' 
        OR name ILIKE '%data science%' 
        OR name ILIKE '%исследователь данных%' 
        OR name ILIKE '%машинн%обучен%%'
        OR name ILIKE '%machine learning%'
        OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%' and name not Ilike '%QML%'))
        AND key_skills ILIKE '%python%'
    
'''

In [111]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df
print(f'Количество вакансий c ключевым навыком Python: {df.iloc[0,0]}.')

Количество вакансий c ключевым навыком Python: 351.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.


Посчитать количество определённых символов в строке с помощью SQL можно следующим образом: из длины исходной строки вычесть длину строки, в которой все вхождения этого символа заменены на пустую строку ('').

Ключевые навыки разделены символами табуляции. В SQL этот символ обозначается как CHR(9).

22:58
Если ключевые навыки не заполнены, то такую вакансию не рассматриваем для данного расчета, так как незаполненное поле ключевых навыков не равносильно тому, что у этой вакансии ноль ключевых навыков. Скорее всего они будут указаны в тексте вакансии.

In [112]:
# текст запроса
query_6_5 = f'''
 
    SELECT /*Запрос выводит среднее количество ключевых навыков в вакансиях для DS.*/    
       round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2)
    FROM  
        VACANCIES 
    WHERE 
        (name ILIKE '%data scientist%' 
        OR name ILIKE '%data science%' 
        OR name ILIKE '%исследователь данных%' 
        OR name ILIKE '%машинн%обучен%%'
        OR name ILIKE '%machine learning%'
        OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%'))
        and key_skills IS NOT NULL 
        
''' 

In [113]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
print(f'В среднем в вакансиях для DS указывают ключевых навыков: {df.iloc[0,0]}.')

В среднем в вакансиях для DS указывают ключевых навыков: 6.41.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [114]:
# текст запроса
query_6_6 = f'''
    with x as /*Формируем таблицу с необходимым опытом работы, нижним и верхним уровкнем ЗП для дата-сайентиста*/
        (
        SELECT
            experience,
            salary_from,
            salary_to
        FROM
            VACANCIES
        WHERE 
            (name ILIKE '%data scientist%' 
            OR name ILIKE '%data science%' 
            OR name ILIKE '%исследователь данных%' 
            OR name ILIKE '%машинн%обучен%%'
            OR name ILIKE '%machine learning%'
            OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')) 
            AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
        )
    SELECT /*Запрос выводит усредненную ЗП для специалистов с опытом работы от 3 до 6 лет.*/   
       experience,
       ROUND(AVG((COALESCE(salary_from,salary_to)+COALESCE(salary_to,salary_from))/2))::int avg_salary
    FROM x
    WHERE experience = 'От 3 до 6 лет'	
    GROUP BY experience

'''


In [115]:
# результат запроса 
df = pd.read_sql_query(query_6_6, connection)
print(f'Дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать на зарплату: {df.iloc[0,1]} руб.')

Дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать на зарплату: 243115 руб.


# Выводы по предметному анализу

- Количество вакансий имеющих отношение к данным: 1771.

- Количество подходящих вакансий для начинающего дата-сайентиста: 51. 
  Это можно объяснить тем что профессия довольно новая. 

- Количество вакансий c ключевым навыком SQL или postgres: 229. 
  Показывает востребованность умения работы с базами данных.
  
- Количество вакансий c ключевым навыком Python: 357. 
  Ввиду популярности языка для многих работодателей владение им является преимуществом (требуется предобработка информации). 

- В среднем в вакансиях для DS указывают ключевых навыков: 6.41. 
  Это указываем на желание работодателя получить специалиста с несколькими ключевыми навыками.

- Усредненная зарплата начинающего дата-сайентиста с увеличением опыта работы может вырасти более чем в три раза и составить: 243115 руб.

# Дополнительные исследования по проекту

In [116]:
# Код для получения среднего количества ключевых навыков для начинающего дата-сайентиста.
query_7_0 = f'''
 
    SELECT /*Запрос выводит среднее количество ключевых навыков в вакансиях для DS.*/    
       round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2)
    FROM  
        VACANCIES 
    WHERE 
        (name ILIKE '%data scientist%' 
        OR name ILIKE '%data science%' 
        OR name ILIKE '%исследователь данных%' 
        OR name ILIKE '%машинн%обучен%%'
        OR name ILIKE '%machine learning%'
        OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%'))
        AND key_skills IS NOT NULL    
        AND (name ILIKE '%junior%' 
            OR experience ='Нет опыта'
            OR schedule = 'Стажировка') 
            
''' 

In [117]:
# результат запроса

df = pd.read_sql_query(query_7_0, connection)
print(f'Среднее количество ключевых навыков для начинающего дата-сайентиста: {df.iloc[0,0]}.')

Среднее количество ключевых навыков для начинающего дата-сайентиста: 7.27.


In [118]:
# Количество работодателей у которых есть хотябы одна вакансия.
query_7_1 = f'''

select 
    count(distinct employer_id)
from
    VACANCIES
    
'''

In [119]:
# результат запроса
df = pd.read_sql_query(query_7_1, connection)
print(f'Количество работодателей у которых есть хотябы одна вакансия: {df.iloc[0,0]}.')
print(f'Количество работодателей у которых есть хотябы одна вакансия: \
{round((df.iloc[0,0]/pd.read_sql_query(query_3_2, connection).iloc[0,0]*100),2)}%.')

Количество работодателей у которых есть хотябы одна вакансия: 14906.
Количество работодателей у которых есть хотябы одна вакансия: 63.43%.


In [120]:
# Получаем количество работодателей у которых нет вакансий.
query_7_2 = f'''
    
    SELECT
        COUNT(id) - (select 
                        COUNT(distinct employer_id) 
                    FROM
                        VACANCIES
                    ) AS emp_without_vac
    FROM
        EMPLOYERS

'''

In [121]:
# результат запроса
df = pd.read_sql_query(query_7_2, connection)
print(f'Количество работодателей у которых нет вакансий: {df.iloc[0,0]}.')
print(f'Количество работодателей у которых нет вакансий: \
{round((df.iloc[0,0]/pd.read_sql_query(query_3_2, connection).iloc[0,0]*100),2)}%.')

Количество работодателей у которых нет вакансий: 8595.
Количество работодателей у которых нет вакансий: 36.57%.


In [122]:
# Получаем количество работодателей не указывающих размер зарплаты.
query_7_3 = f'''

    SELECT 
        COUNT (distinct employer_id)    
    FROM 
        VACANCIES 
    WHERE 
        salary_from IS NOT NULL AND salary_to IS NOT NULL

'''

In [123]:
# результат запроса
df = pd.read_sql_query(query_7_3, connection)
print(f'Количество работодателей не указывающих размер зарплаты: {df.iloc[0,0]}.')
print(f'Доля работодателей не указывающих размер зарплаты: \
{round((df.iloc[0,0]/pd.read_sql_query(query_3_2, connection).iloc[0,0]*100),2)}%.')

Количество работодателей не указывающих размер зарплаты: 5500.
Доля работодателей не указывающих размер зарплаты: 23.4%.


In [124]:
# Получаем количество вакансий с не указанным размером зарплаты
query_7_4 = f'''

    SELECT COUNT (id)    
    FROM VACANCIES 
    WHERE salary_from IS NULL AND salary_to IS NULL

'''


In [125]:
# результат запроса
df = pd.read_sql_query(query_7_4, connection)
print(f'Количество вакансий с не указанным размером зарплаты: {df.iloc[0,0]}.')
print(f'Доля вакансий с не указанным размером зарплаты: \
{round((df.iloc[0,0]/pd.read_sql_query(query_3_1, connection).iloc[0,0]*100),2)}%.')

Количество вакансий с не указанным размером зарплаты: 25124.
Доля вакансий с не указанным размером зарплаты: 51.07%.


In [126]:
# Количество работодателей зарегистрированных в наукоградах.
science_city = []
URL = 'https://ru.wikipedia.org/wiki/Наукограды_России'
response = requests.get(URL)
page = BeautifulSoup(response.text, 'html.parser')
# получаем строку с тегом 'table' класс 'standard' содержащую всю информацию в таблице
temp_str = str(page.find('table', class_='wikitable')).split('title="') 
# добавляем в список только названия городов
for i in temp_str[1:]:
    if 'область' in i or 'край' in i:
        continue
    temp = i.split('">')[0].split(' ')[0]        
    science_city.append(temp)
    
query_7_5 = f'''

    SELECT /*Количество работодателей со сферой деятельности - Разработка программного обеспечения. */
        COUNT(em.employer_id)
    FROM EMPLOYERS e 
        left join AREAS a on e.area = a.id
        LEFT JOIN EMPLOYERS_INDUSTRIES em ON e.id = em.employer_id
        LEFT JOIN INDUSTRIES i ON i.id = em.industry_id 
    WHERE i.name = 'Разработка программного обеспечения' and a.name in {tuple(science_city)} 
      
'''

In [127]:
df = pd.read_sql_query(query_7_5, connection)
print(f'В наукоградах присутствуют {df.iloc[0,0]} работодателя, со сферой деятельности - Разработка программного обеспечения.')

В наукоградах присутствуют 1337 работодателя, со сферой деятельности - Разработка программного обеспечения.


In [128]:
# Формируем таблицу с работодателями отсортированных по общему количеству вакансий,
# добавлены средняя ЗП по вакансиям работодателя, а также количество вакансий и 
# усредненная ЗП  в городах-миллионниках соответственно работодателю..

list_m_city = []
URL = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(URL)
page = BeautifulSoup(response.text, 'html.parser')
# получаем строку с тегом 'table' класс 'standard' содержащую всю информацию в таблице
temp_str = str(page.find('table', class_='standard')).split('title="') 
# добавляем в список только названия городов
for i in temp_str[3:]:
    temp = i.split('">')[0]
    list_m_city.append(temp)

query_7_6 = f'''

WITH temp AS
(
   WITH temp_table_one AS /*Запрос выдает название работодателя, его id, общее количество его вакансий и усредненную м,,,,,, */
   (
      SELECT 
         e.name AS company_name,
         e.id AS id,
         COUNT(v.id) AS total_vacancies,         
         ROUND(AVG((COALESCE(salary_from,salary_to)+COALESCE(salary_to,salary_from))/2))::int avg_salary         
      FROM VACANCIES v
      JOIN EMPLOYERS e ON v.employer_id = e.id       
      GROUP BY e.name, e.id
   ),   
   temp_table_two AS /*Запрос выдает количество вакансий и среднюю ЗП в городах-миллионерах*/
   (
      SELECT 
         e.id AS id,
         COUNT(v.id) AS vacancies,
         ROUND(AVG((COALESCE(salary_from,salary_to)+COALESCE(salary_to,salary_from))/2))::int avg_salary_m_city
      FROM VACANCIES v
      JOIN EMPLOYERS e ON v.employer_id = e.id
      LEFT JOIN AREAS a ON v.area_id = a.id
      WHERE a.name IN {tuple(list_m_city)}         
      GROUP BY e.id
   ) 
   SELECT 
      t1.company_name,
      --t1.id,
      t1.total_vacancies,
      t1.avg_salary,   
      t2.vacancies vacancies_m_city,
      t2.avg_salary_m_city
   FROM temp_table_one t1
   LEFT JOIN temp_table_two t2 ON t1.id = t2.id   
)

select *
from temp
order by total_vacancies desc 

'''

In [129]:
# результат запроса
df = pd.read_sql_query(query_7_6, connection)
df.head(10)	

,company_name,total_vacancies,avg_salary,vacancies_m_city,avg_salary_m_city
0,Яндекс,1933,25884.0,485.0,31702.0
1,Ростелеком,491,46980.0,214.0,56945.0
2,Тинькофф,444,50143.0,285.0,50167.0
3,СБЕР,428,153847.0,364.0,159099.0
4,Газпром нефть,331,56000.0,245.0,42000.0
5,ИК СИБИНТЕК,327,53434.0,147.0,60857.0
6,МТС,292,64521.0,231.0,67260.0
7,DataArt,247,NaN,NaN,NaN
8,Совкомбанк Технологии,204,42871.0,143.0,43125.0
9,Первый Бит,176,92324.0,104.0,90063.0


# Выводы из дополнительного исследования:
- 5500 работодателей предпочитают не указывать размер зарплаты, их доля от общего числа работодателей составляет 23.4%;

- 25124 вакансии размещены без указания размера зарплаты, их доля от общего числа составляет 51.07%;
Это можно объяснить тем, что зарплата, в этом случае, определяется индивидуально на собеседовании исходя из комппетенций соискателя. 

- Количество работодателей у которых есть хотябы одна вакансия - 14906, что составляет 63.43% от общего числа.

- По результату проведенного исследования видно что топ-10 компаний по количеству вакансий - крупные IT компании, банки и Газпром нефть. Значительная доля вакансий приходится на города-миллионники, исключая компанию DataArt, вакансии которой не имеющют привязки к региону.

- Разработкой программного обеспечения занимаются: 3553 работодателя из которых 1337 pарегистрированны в наукоградах.
  Этому видимо способствует налоговая политика, инфраструктура, господдержка и насыщеность рынка труда специалистами.

***

# Общий вывод по проекту

Из проведенного исследования можно заключить, что с ростом уровня компитенций соискателя, как и следовало ожидать, он может претендовать на более высокооплачиваемые вакансии. 

Работодатели, в основном, ищут кандидатов сопытом от 1 года до 3 лет (26152) и от 3 лет до 6 лет (14511) что говорит о востребованности этих специалистов на рынке труда. Кандидатов без опыта работы хотят привлечь с на меньшее количество вакансий -7197, что видимо обусловленно необхдимостью дополнительного обучения. Меньше всего вакансий для кандидатов сопытом от 6 лет - работодатели ищут высококлассных специалистов.

Можно отметить, что для многих работодателей средний уровень зарплаты по вакансиям крупных городах выше усредненной общей зарплаты - видимо основные вакансии расположены в городах с развитой инфраструктурой и более высоким уровнем жизни где, для привлечения кандатов, требуется более высокий уровень оплаты труда.

Начинающему дата-сайентистау необходимо освоить более 7.27 ключевых навыков для успешного прохождения собеседования, тогда как в общем ключевых навыков по профессии в среднем требуется 6.41. Видимо это говорит о том, что вакансии для более опытных соискателей проразумевают наличие "базовых" компетенций. 

Для дальнейшего анализа интересно посмотреть взаимосвязи в разрезе опыта, количестве ключевых навыков, уровня зарплаты по крупным IT команиям.